In [ ]:
import pypsa
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

plt.style.use("ggplot")
import pycountry
import json
import warnings

warnings.filterwarnings("ignore")

In [ ]:
available_models = {
    "model_1": "elec_s_37_ec_lv1.0_.nc",
    "model_2": "elec_s_37_ec_lv1.0_3H_withUC.nc",
    "model_3": "elec_s_37_ec_lv1.0_Co2L-noUC-noCo2price.nc",
    "model_4": "elec_s_37_ec_lv1.0_Ep.nc",
    "model_5": "elec_s_37_ec_lv1.0_Ep_new.nc",
}

model_choice = "model_5"

data_path = Path.cwd() / ".." / ".."
model_path = data_path / available_models[model_choice]

with open(data_path / "generation_data" / "generation_mapper_pypsa.json", "r") as f:
    pypsa_generation_mapper = json.load(f)

plot_path = data_path / "plots" / available_models[model_choice][:-3]

In [ ]:
os.mkdir(data_path / "plots" / available_models[model_choice][:-3])

In [ ]:
n = pypsa.Network(str(model_path))

In [ ]:
def intersection(alist, blist):
    total_list = list()
    for val in alist:
        if val in blist:
            total_list.append(val)
    return total_list

In [ ]:
pypsa_generation_mapper

In [ ]:
color_mapper = pd.read_csv("color_mapper.csv", index_col=0).iloc[:, 0]
color_mapper.loc["Others"] = "#D3D3D3"
color_mapper.loc["Storage Charge"] = "#51dbcc"
color_mapper.loc["Storage Discharge"] = "#51dbcc"

In [ ]:
color_mapper

In [ ]:
countries = set([col[:2] for col in n.generators_t.p.columns])
gen = set([col[6:] for col in n.generators_t.p.columns])

for i, country in enumerate(countries):
    df = pd.DataFrame(index=n.generators_t.p.index)
    # country_generation = [col for col in n.generators_t.p.columns if col.startswith(country)]
    country_generation = n.generators.loc[n.generators.bus.str.contains(country)]

    for key, gens in pypsa_generation_mapper.items():
        # curr_gen = country_generation.loc[
        #     (country_generation.carrier.str.contains(tech) for tech in gens).astype(bool)].index
        curr_gen = country_generation.loc[
            country_generation.carrier.apply(lambda carr: carr in gens)
        ].index

        if len(curr_gen):
            df[key] = n.generators_t.p[curr_gen].mean(axis=1)
        else:
            df[key] = np.zeros(len(df))

    df.to_csv(data_path / "pypsa_data" / (country + ".csv"))

In [ ]:
total_inflow_cols = [
    "Solar",
    "Wind Onshore",
    "Nuclear",
    "Lignite",
    "Inflow Lines",
    "Inflow Links",
    "Wind Offshore",
    "Biomass",
    "Run of River",
    "Hydro",
    "Hard Coal",
    "Gas",
    "Oil",
]
total_outflow_cols = ["Outflow Links", "Outflow Lines", "Storage Charge"]

In [ ]:
total_inflow_set = set()
total_outflow_set = set()

In [ ]:
import seaborn as sns
from sklearn.metrics import mean_absolute_error

index = n.generators_t.p.index

pypsa_total_inflow = pd.DataFrame(
    np.zeros((len(index), len(total_inflow_cols))),
    index=index,
    columns=total_inflow_cols,
)
entsoe_df = pd.read_csv(
    data_path / "harmonised_generation_data" / ("prepared_DE.csv"),
    parse_dates=True,
    index_col=0,
)
entsoe_total_inflow = pd.DataFrame(
    np.zeros((len(entsoe_df), len(total_inflow_cols))),
    index=entsoe_df.index,
    columns=total_inflow_cols,
)
pypsa_total_outflow = pd.DataFrame(
    np.zeros((len(index), len(total_outflow_cols))),
    index=index,
    columns=total_outflow_cols,
)
total_load = pd.Series(index=index)

for num, country in enumerate(os.listdir(data_path / "pypsa_data")):
    # country = "DE.csv"
    cc = country[:2]

    country_buses = np.unique(
        n.generators.loc[n.generators.bus.str.contains(cc)].bus.values
    )
    print(f"Buses for country {country[:-4]}: ", country_buses)

    if not len(country_buses) == 1:
        print("Current implementation is for one bus per country")
        print(f"Skipping!")
        continue

    bus = country_buses[0]

    """    
    pypsa_df = pd.read_csv(data_path / "pypsa_data" / country, parse_dates=True, index_col=0)
    """
    try:
        entsoe_df = pd.read_csv(
            data_path / "harmonised_generation_data" / ("prepared_" + country),
            parse_dates=True,
            index_col=0,
        )

        entsoe_df.columns = [col[:-6] for col in entsoe_df.columns]
        entsoe_df = entsoe_df.iloc[1:]
        entsoe_df = entsoe_df.multiply(1e-3)
    except FileNotFoundError:
        continue

    fig, axs = plt.subplots(4, 3, figsize=(20, 20))

    axs[0, 0].set_title(pycountry.countries.get(alpha_2=country[:2]).name)

    start = pd.Timestamp("2019-01-01")  # for small time frame
    end = pd.Timestamp("2019-01-14")
    coarse_freq = "3d"

    num_techs_shown = 6

    energy_inflow = pd.DataFrame(index=n.loads_t.p_set.index)
    energy_outflow = pd.DataFrame(index=n.loads_t.p_set.index)

    # add generation
    country_gen = n.generators.loc[n.generators.bus == bus]

    for tech, pypsa_carrier in pypsa_generation_mapper.items():
        gens = country_gen.loc[
            country_gen.carrier.apply(lambda c: c in pypsa_carrier)
        ].index
        energy_inflow[tech] = n.generators_t.p[gens].sum(axis=1)

    # add inflows from lines
    lines0 = n.lines.loc[n.lines.bus0 == bus].index
    lines1 = n.lines.loc[n.lines.bus1 == bus].index

    lines_flow = np.zeros(energy_inflow.shape[0])
    if not lines0.empty:
        lines_flow = -n.lines_t.p0[lines0].sum(axis=1)

    if not lines1.empty:
        lines_flow -= n.lines_t.p1[lines1].sum(axis=1)

    energy_inflow["Inflow Lines"] = np.maximum(np.zeros_like(lines_flow), lines_flow)
    energy_outflow["Outflow Lines"] = np.minimum(np.zeros_like(lines_flow), lines_flow)

    # add inflows from links
    links0 = n.links.loc[n.links.bus0 == bus].index
    links1 = n.links.loc[n.links.bus1 == bus].index

    links_flow = np.zeros(energy_inflow.shape[0])
    if not links0.empty:
        links_flow = (
            -n.links_t.p0[links0]
            .multiply(n.links.loc[links0, "efficiency"])
            .sum(axis=1)
        )

    if not links1.empty:
        links_flow -= (
            n.links_t.p1[links1].multiply(n.links.loc[links1, "efficiency"]).sum(axis=1)
        )

    energy_inflow["Inflow Links"] = np.maximum(np.zeros_like(links_flow), links_flow)
    energy_outflow["Outflow Links"] = np.minimum(np.zeros_like(links_flow), links_flow)

    storage = n.storage_units.loc[n.storage_units.bus == bus].index
    if not storage.empty:
        storage_p = n.storage_units_t.p[storage].sum(axis=1).values
        # energy_inflow["Storage Discharge"] = np.maximum(np.zeros_like(links_flow), storage_p)
        energy_inflow["Hydro"] = energy_inflow["Hydro"].values + np.maximum(
            np.zeros_like(links_flow), storage_p
        )
        energy_outflow["Storage Charge"] = np.minimum(
            np.zeros_like(links_flow), storage_p
        )

    energy_inflow = energy_inflow.iloc[:-1].multiply(1e-3)
    energy_outflow = energy_outflow.iloc[:-1].multiply(1e-3)
    load = n.loads_t.p_set[bus].iloc[:-1].multiply(1e-3)

    total_load = total_load.loc[load.index]
    total_load = total_load + load

    pypsa_total_inflow = pypsa_total_inflow.loc[energy_inflow.index]
    pypsa_total_inflow[energy_inflow.columns] = (
        pypsa_total_inflow[energy_inflow.columns] + energy_inflow
    )

    pypsa_total_outflow = pypsa_total_outflow.loc[energy_outflow.index]
    pypsa_total_outflow[energy_outflow.columns] = (
        pypsa_total_outflow[energy_outflow.columns] + energy_outflow
    )

    entsoe_total_inflow = entsoe_total_inflow.loc[entsoe_df.index]
    entsoe_total_inflow[entsoe_df.columns] = entsoe_total_inflow[
        entsoe_df.columns
    ] + entsoe_df.fillna(0.0)

    show_techs = (
        energy_inflow.sum()
        .sort_values(ascending=False)
        .iloc[:num_techs_shown]
        .index.tolist()
    )
    others = (
        energy_inflow.sum()
        .sort_values(ascending=False)
        .iloc[num_techs_shown:]
        .index.tolist()
    )
    # show_techs = entsoe_df.sum().sort_values(ascending=False).iloc[:num_techs_shown].index.tolist()

    show_techs = intersection(show_techs, entsoe_df.columns.tolist())
    entsoe_df["Others"] = entsoe_df.drop(columns=show_techs).sum(axis=1)

    # entsoe_df[show_techs + ["Others"]].loc[start:end].plot.area(ax=axs[0,0])
    index = load.loc[start:end].index

    entsoe_df.index = load.index

    energy_inflow["Others"] = energy_inflow.drop(columns=show_techs).sum(axis=1)

    # plot timeframe
    axs[0, 0].plot(
        index,
        load.loc[index].values,
        linestyle="--",
        color="k",
        linewidth=2,
        label="PyPSA Load",
    )
    axs[0, 1].plot(
        index, load.loc[index].values, linestyle="--", color="k", linewidth=2
    )

    axs[0, 1].stackplot(
        index,
        *[energy_inflow[col].loc[index].values for col in show_techs + ["Others"]],
        colors=color_mapper.loc[show_techs + ["Others"]].tolist(),
    )
    axs[0, 1].stackplot(
        index,
        *[energy_outflow[col].loc[index].values for col in energy_outflow.columns],
        colors=color_mapper.loc[energy_outflow.columns].tolist(),
        labels=energy_outflow.columns,
    )

    axs[0, 0].stackplot(
        index,
        *[entsoe_df[col].loc[index].values for col in show_techs + ["Others"]],
        labels=show_techs + ["Others"],
        colors=color_mapper.loc[show_techs + ["Others"]].tolist(),
    )
    axs[0, 1].plot(
        index,
        energy_inflow.loc[index][show_techs + ["Others"]].sum(axis=1).values
        + energy_outflow.loc[index].sum(axis=1).values,
        color="brown",
        linestyle=":",
        linewidth=2,
        label="Accum Gen",
    )

    axs[0, 0].legend()
    axs[0, 1].legend()

    # plot whole year

    index = load.resample(coarse_freq).mean().index

    axs[1, 0].plot(
        index,
        load.resample(coarse_freq).mean().values,
        linestyle="--",
        color="k",
        linewidth=2,
        label="PyPSA Load",
    )
    axs[1, 1].plot(
        index,
        load.resample(coarse_freq).mean().values,
        linestyle="--",
        color="k",
        linewidth=2,
    )

    axs[1, 1].stackplot(
        index,
        *[
            energy_inflow[col].resample(coarse_freq).mean().values
            for col in show_techs + ["Others"]
        ],
        colors=color_mapper.loc[show_techs + ["Others"]].tolist(),
    )
    axs[1, 1].stackplot(
        index,
        *[
            energy_outflow[col].resample(coarse_freq).mean().values
            for col in energy_outflow.columns
        ],
        colors=color_mapper.loc[energy_outflow.columns].tolist(),
        labels=energy_outflow.columns,
    )

    axs[1, 0].stackplot(
        index,
        *[
            entsoe_df[col].resample(coarse_freq).mean().values
            for col in show_techs + ["Others"]
        ],
        colors=color_mapper.loc[show_techs + ["Others"]].tolist(),
        labels=show_techs + ["Others"],
    )

    axs[1, 1].plot(
        index,
        energy_inflow.resample(coarse_freq)
        .mean()[show_techs + ["Others"]]
        .sum(axis=1)
        .values
        + energy_outflow.resample(coarse_freq).mean().sum(axis=1).values,
        color="brown",
        linestyle=":",
        linewidth=2,
        label="Accum Gen",
    )

    axs[1, 0].legend()
    axs[1, 1].legend()

    y_min = pd.concat([energy_outflow.sum(axis=1)]).min()
    y_max = pd.concat(
        [energy_inflow.sum(axis=1), entsoe_df.sum(axis=1)], ignore_index=True
    ).max()

    for ax in axs[:2, :2].flatten():
        ax.set_ylim(y_min, y_max)
        ax.set_ylim(y_min, y_max)

    axs[0, 0].set_ylabel("ENTSOE Gen and PyPSA Load [GW]")
    axs[0, 1].set_ylabel("PyPSA Gen and Load [GW]")
    axs[1, 0].set_ylabel("ENTSOE Gen and PyPSA Load [GW]")
    axs[1, 1].set_ylabel("PyPSA Gen and Load [GW]")
    axs[2, 0].set_ylabel("ENTSOE Gen and PyPSA Load [GW]")
    axs[2, 1].set_ylabel("PyPSA Gen and Load [GW]")

    # -------------------------- electricity prices comparison ----------------------------------
    prices_col = [
        col for col in n.buses_t.marginal_price.columns if col.startswith(country[:2])
    ]
    pypsa_prices = n.buses_t.marginal_price[prices_col].mean(axis=1)

    full_index = pypsa_prices.index

    coarse_pypsa_prices = pypsa_prices.resample(coarse_freq).mean()
    pypsa_prices = pypsa_prices.loc[start:end]

    axs[0, 2].plot(
        pypsa_prices.index, pypsa_prices.values, label="PyPSA prices", color="royalblue"
    )
    axs[1, 2].plot(
        coarse_pypsa_prices.index,
        coarse_pypsa_prices.values,
        label="PyPSA prices",
        color="royalblue",
    )

    try:
        entsoe_prices = pd.read_csv(
            data_path / "price_data" / country,
            index_col=0,
            parse_dates=True,
        ).iloc[:-1]

        def make_tz_time(time):
            return pd.Timestamp(time).tz_convert("utc")

        # entsoe_prices.index = pd.Series(entsoe_prices.index).apply(lambda time: make_tz_time(time))
        entsoe_prices.index = full_index
        mean_abs_error = mean_absolute_error(
            entsoe_prices.values,
            n.buses_t.marginal_price[prices_col].mean(axis=1).values,
        )

        coarse_prices = entsoe_prices.resample(coarse_freq).mean()
        entsoe_prices = entsoe_prices.loc[start:end]

        axs[0, 2].plot(
            entsoe_prices.index,
            entsoe_prices.values,
            label="ENTSOE prices",
            color="darkred",
        )
        axs[1, 2].plot(
            coarse_prices.index,
            coarse_prices.values,
            label="ENTSOE prices",
            color="darkred",
        )

    except FileNotFoundError:
        mean_abs_error = None
        pass

    upper_lim = pd.concat((entsoe_prices, pypsa_prices), axis=0).max().max()
    for ax in axs[:2, 2]:
        ax.set_ylim(0, upper_lim)
        ax.set_ylabel("Electricty Prices [Euro/MWh]")
        ax.legend()

    if not mean_abs_error is None:
        axs[1, -1].set_title(f"Mean Abs Error: {np.around(mean_abs_error, decimals=2)}")

    # remaining_cols = energy_inflow.drop(column    s=show_techs+["Others"]).columns.tolist()
    # axs[1,0].set_title(f"Others: {remaining_cols}")

    # ------------------------------- duration curves ------------------------------

    entsoe_ddf = entsoe_df[show_techs + ["Others"]].reset_index(drop=True)

    entsoe_ddf = pd.concat(
        [
            entsoe_ddf[col].sort_values(ascending=False).reset_index(drop=True)
            for col in entsoe_ddf.columns
        ],
        axis=1,
    )

    axs[2, 0].stackplot(
        range(len(entsoe_ddf)),
        *[entsoe_ddf[col].values for col in entsoe_ddf.columns],
        colors=color_mapper.loc[entsoe_ddf.columns].tolist(),
        labels=entsoe_ddf.columns,
    )

    pypsa_ddf = energy_inflow[show_techs + ["Others"]].reset_index(drop=True)
    pypsa_ddf = pd.concat(
        [
            pypsa_ddf[col].sort_values(ascending=False).reset_index(drop=True)
            for col in pypsa_ddf.columns
        ],
        axis=1,
    )

    axs[2, 1].stackplot(
        range(len(pypsa_ddf)),
        *[pypsa_ddf[col].values for col in pypsa_ddf.columns],
        colors=color_mapper.loc[pypsa_ddf.columns].tolist(),
        labels=pypsa_ddf.columns,
    )

    ylim_max = max([pypsa_ddf.max(axis=0).sum(), entsoe_ddf.max(axis=0).sum()])

    pypsa_ddf = energy_outflow.reset_index(drop=True)
    pypsa_ddf = pd.concat(
        [
            pypsa_ddf[col].sort_values(ascending=True).reset_index(drop=True)
            for col in pypsa_ddf.columns
        ],
        axis=1,
    )

    axs[2, 1].stackplot(
        range(len(pypsa_ddf)),
        *[pypsa_ddf[col].values for col in pypsa_ddf.columns],
        colors=color_mapper.loc[pypsa_ddf.columns].tolist(),
        labels=energy_outflow.columns,
    )

    ylim_min = energy_outflow.min(axis=0).sum()

    for ax in axs[2, :2]:
        ax.legend()
        ax.set_ylim(ylim_min, ylim_max)

    pypsa_totals = pd.concat(
        [energy_inflow[show_techs + ["Others"]], energy_outflow], axis=1
    ).sum()

    entsoe_totals = entsoe_df.sum()
    totals = pd.DataFrame(index=pypsa_totals.index)

    for tech in pypsa_totals.index:
        if tech not in entsoe_totals.index:
            entsoe_totals.loc[tech] = 0.0

    totals["Pypsa"] = pypsa_totals
    totals["Entsoe"] = entsoe_totals
    totals["Technology"] = totals.index

    totals = pd.concat(
        [
            pd.DataFrame(
                {
                    "Source": ["PyPSA" for _ in range(len(pypsa_totals))],
                    "Technology": pypsa_totals.index,
                    "Total Generation": pypsa_totals.values,
                }
            ),
            pd.DataFrame(
                {
                    "Source": ["ENTSO-E" for _ in range(len(entsoe_totals))],
                    "Technology": entsoe_totals.index,
                    "Total Generation": entsoe_totals.values,
                }
            ),
        ],
        axis=0,
    )

    sns.barplot(
        data=totals,
        x="Technology",
        y="Total Generation",
        hue="Source",
        ax=axs[2, 2],
        palette="dark",
        alpha=0.6,
        edgecolor="k",
    )

    axs[2, 0].set_xlabel("Hours")
    axs[2, 1].set_xlabel("Hours")
    axs[2, 2].set_ylabel("Total Generation [GWh]")
    axs[2, 2].set_xticks(
        axs[2, 2].get_xticks(), axs[2, 2].get_xticklabels(), rotation=45, ha="right"
    )

    corrs = (
        energy_inflow.corrwith(entsoe_df)
        .drop(index="Others")
        .dropna()
        .sort_values(ascending=False)
    )

    for col, ax in zip(corrs.index[:2].tolist() + [corrs.index[-1]], axs[3]):
        ax.scatter(
            entsoe_df[col].values,
            energy_inflow[col].values,
            color="darkred",
            alpha=0.5,
            s=20,
            edgecolor="k",
        )
        ax.set_title(f"{col}; Pearson Corr {np.around(corrs.loc[col], decimals=4)}")
        ax.set_xlabel("ENTSO-E Generation [GW]")
        ax.set_ylabel("PyPSA-Eur Generation [GW]")

    for ax in axs[:2].flatten():
        ax.set_xlabel("Datetime")

    plt.tight_layout()
    plt.savefig(plot_path / (cc + ".pdf"))

    plt.show()

In [ ]:
# pypsa_total_inflow.to_csv("total_inflow_pypsa.csv")
# pypsa_total_outflow.to_csv("total_outflow_pypsa.csv")
# entsoe_total_inflow.to_csv("total_inflow_entsoe.csv")
# total_load.to_csv("total_load.csv")

In [ ]:
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")
import seaborn as sns
import numpy as np

entsoe_total_inflow = pd.read_csv(
    "total_inflow_entsoe.csv", index_col=0, parse_dates=True
)
pypsa_total_inflow = pd.read_csv(
    "total_inflow_pypsa.csv", index_col=0, parse_dates=True
)
pypsa_total_outflow = pd.read_csv(
    "total_outflow_pypsa.csv", index_col=0, parse_dates=True
)
total_load = pd.read_csv("total_load.csv", index_col=0, parse_dates=True)

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# pypsa_totals = pd.concat([pypsa_total_inflow, pypsa_total_outflow], axis=1).sum() * 1e-3
pypsa_totals = (
    pypsa_total_inflow.drop(columns=["Inflow Lines", "Inflow Links"]).sum() * 1e-3
)

entsoe_totals = (
    entsoe_total_inflow.drop(columns=["Inflow Lines", "Inflow Links"]).sum() * 1e-3
)
totals = pd.DataFrame(index=pypsa_totals.index)

for tech in pypsa_totals.index:
    if tech not in entsoe_totals.index:
        entsoe_totals.loc[tech] = 0.0

totals["Pypsa"] = pypsa_totals
totals["Entsoe"] = entsoe_totals
totals["Technology"] = totals.index

totals = pd.concat(
    [
        pd.DataFrame(
            {
                "Source": ["PyPSA" for _ in range(len(pypsa_totals))],
                "Technology": pypsa_totals.index,
                "Total Generation": pypsa_totals.values,
            }
        ),
        pd.DataFrame(
            {
                "Source": ["ENTSO-E" for _ in range(len(entsoe_totals))],
                "Technology": entsoe_totals.index,
                "Total Generation": entsoe_totals.values,
            }
        ),
    ],
    axis=0,
)


sns.barplot(
    data=totals,
    x="Technology",
    y="Total Generation",
    hue="Source",
    ax=ax,
    palette="dark",
    alpha=0.6,
    edgecolor="k",
)
ax.set_ylabel("Total Generation [TWh]")
ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha="right")
plt.savefig(plot_path / "EuropeTotalGeneration.pdf")
plt.show()

pypsa_total_inflow = pypsa_total_inflow.drop(columns=["Inflow Links", "Inflow Lines"])
pypsa_total_outflow = pypsa_total_outflow.drop(
    columns=["Outflow Links", "Outflow Lines"]
)
entsoe_total_inflow = entsoe_total_inflow.drop(columns=["Inflow Links", "Inflow Lines"])

start = pd.Timestamp("2019-01-01")  # for small time frame
end = pd.Timestamp("2019-01-14")
coarse_freq = "3d"

index = load.loc[start:end].index
cols = pypsa_total_inflow.std(axis=0).sort_values(ascending=True).index
cols_out = pypsa_total_outflow.std(axis=0).sort_values(ascending=False).index

fig, axs = plt.subplots(4, 2, figsize=(20, 30))

axs[0, 0].stackplot(
    index,
    *[entsoe_total_inflow[col].loc[start:end].values for col in cols],
    colors=color_mapper.loc[cols].tolist(),
)

axs[0, 1].stackplot(
    index,
    *[pypsa_total_inflow[col].loc[start:end].values for col in cols],
    colors=color_mapper.loc[cols].tolist(),
)
axs[0, 1].stackplot(
    index,
    *[pypsa_total_outflow[col].loc[start:end].values for col in cols_out],
    colors=color_mapper.loc[cols_out].tolist(),
)

entsoe_total_inflow = entsoe_total_inflow.resample(coarse_freq).mean()
pypsa_total_inflow = pypsa_total_inflow.resample(coarse_freq).mean()
pypsa_total_outflow = pypsa_total_outflow.resample(coarse_freq).mean()

index = pypsa_total_inflow.index

axs[1, 0].stackplot(
    index,
    *[entsoe_total_inflow[col].values for col in cols],
    colors=color_mapper.loc[cols].tolist(),
)

axs[1, 1].stackplot(
    index,
    *[pypsa_total_inflow[col].values for col in cols],
    colors=color_mapper.loc[cols].tolist(),
)
axs[1, 1].stackplot(
    index,
    *[pypsa_total_outflow[col].values for col in cols_out],
    colors=color_mapper.loc[cols_out].tolist(),
)

for ax in axs[:3].flatten():
    ax.set_ylim(-100, 400)


total_entsoe_ddf = pd.concat(
    [
        entsoe_total_inflow[col].sort_values(ascending=False).reset_index(drop=True)
        for col in entsoe_total_inflow.columns
    ],
    axis=1,
)
axs[2, 0].stackplot(
    range(len(total_entsoe_ddf)),
    *[total_entsoe_ddf[col].values for col in total_entsoe_ddf.columns],
    colors=color_mapper.loc[total_entsoe_ddf.columns].tolist(),
    labels=total_entsoe_ddf.columns,
)

total_pypsa_ddf = pd.concat(
    [
        pypsa_total_inflow[col].sort_values(ascending=False).reset_index(drop=True)
        for col in pypsa_total_inflow.columns
    ],
    axis=1,
)
axs[2, 1].stackplot(
    range(len(total_pypsa_ddf)),
    *[total_pypsa_ddf[col].values for col in total_pypsa_ddf.columns],
    colors=color_mapper.loc[total_pypsa_ddf.columns].tolist(),
    labels=total_pypsa_ddf.columns,
)

total_pypsa_ddf = pd.concat(
    [
        pypsa_total_outflow[col].sort_values(ascending=False).reset_index(drop=True)
        for col in pypsa_total_outflow.columns
    ],
    axis=1,
)
axs[2, 1].stackplot(
    range(len(total_pypsa_ddf)),
    *[total_pypsa_ddf[col].values for col in total_pypsa_ddf.columns],
    colors=color_mapper.loc[total_pypsa_ddf.columns].tolist(),
    labels=total_pypsa_ddf.columns,
)
axs[2, 0].legend(
    loc="upper center", bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=5
)

total_prices = (
    n.buses_t.marginal_price.multiply(n.loads_t.p_set)
    .sum(axis=1)
    .divide(n.loads_t.p_set.sum(axis=1))
)

total_entsoe_prices = None

for num, country in tqdm(enumerate(os.listdir(data_path / "pypsa_data"))):
    cc = country[:2]
    country_buses = np.unique(
        n.generators.loc[n.generators.bus.str.contains(cc)].bus.values
    )

    if not len(country_buses) == 1:
        continue

    bus = country_buses[0]

    try:
        entsoe_prices = pd.read_csv(
            data_path / "price_data" / country,
            index_col=0,
            parse_dates=True,
        ).iloc[:-1]
        entsoe_prices.index = n.loads_t.p_set.index

        def make_tz_time(time):
            return pd.Timestamp(time).tz_convert("utc")

    except FileNotFoundError:
        continue

    if total_entsoe_prices is None:
        total_entsoe_prices = pd.Series(
            np.zeros(len(entsoe_prices)), index=entsoe_prices.index
        )

    total_entsoe_prices += entsoe_prices.iloc[:, 0] * n.loads_t.p_set[bus]

total_entsoe_prices /= n.loads_t.p_set.sum(axis=1)

error = np.around(
    mean_absolute_error(total_entsoe_prices.values, total_prices.values), decimals=2
)

axs[3, 0].plot(
    total_prices.loc[start:end].index,
    total_prices.loc[start:end].values,
    label="PyPSA Marginal Price",
)
axs[3, 0].plot(
    total_prices.loc[start:end].index,
    total_entsoe_prices.loc[start:end].values,
    label="ENTSO-E",
)
axs[3, 1].set_title(f"Mean Abs Error {error} [Euro/MWh]")
axs[3, 0].legend()

total_prices = total_prices.resample(coarse_freq).mean()
total_entsoe_prices = total_entsoe_prices.resample(coarse_freq).mean()

axs[3, 1].plot(total_prices.index, total_prices.values, label=f"PyPSA Marginal Price")
axs[3, 1].plot(total_prices.index, total_entsoe_prices.values, label="ENTSO-E Price")

axs[3, 1].legend()

for ax in axs[:3, 0]:
    ax.set_ylabel("ENTSO-E Generation [GWh]")
for ax in axs[:3, 1]:
    ax.set_ylabel("PyPSA Generation [GWh]")
for ax in axs[:2].flatten():
    ax.set_xlabel("Datetime")
for ax in axs[3]:
    ax.set_xlabel("Datetime")
    ax.set_ylabel("Cost of Electricity [Euro/MWh]")
for ax in axs[2]:
    ax.set_xlabel("Hour")

plt.savefig(plot_path / "EuropeDashboard.pdf")
plt.show()